In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
block_size = 8
batch_size = 4 #how many blocks in parallel
learning_rate = 3e-4
#eval_interval = 2500
max_iters = 10000

cuda


In [2]:
with open('wizard-of-oz.txt', 'r', encoding="utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s];
decode = lambda l: "".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    #print(ix)
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device) #pushed to GPU = will be handled there
    return x,y

x, y = get_batch('train')
print("inputs: ")
print(x)

print ("targets: ")
print(y)
    

inputs: 
tensor([[73, 61, 58,  1, 71, 68, 56, 64],
        [57, 58, 67,  1, 68, 59,  1, 73],
        [40, 29,  1,  1,  1,  1,  1,  1],
        [59, 68, 74, 67, 57,  1, 73, 61]], device='cuda:0')
targets: 
tensor([[61, 58,  1, 71, 68, 56, 64, 78],
        [58, 67,  1, 68, 59,  1, 73, 61],
        [29,  1,  1,  1,  1,  1,  1,  1],
        [68, 74, 67, 57,  1, 73, 61, 58]], device='cuda:0')


In [5]:

x = train_data[:block_size]
y = train_data[1:block_size + 1]

print(x)
print(y)

for t in range(block_size):
    context = x[:t+1]
    target=y[t]
    print("when input is", context, "target is", target)

tensor([80,  1,  1, 28, 39, 42, 39, 44])
tensor([ 1,  1, 28, 39, 42, 39, 44, 32])
when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            # batch size, ('time steps' or 'sequence length'), 'channels' = dimensionality of vectors =  vocab_size  
            B, T, C = logits.shape

            #flatten, now where the characters are in their individual sequences doesn't matter
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            #compares the vectors from embedding table to the truth value (truth vectors automatically generated, pytorch expects only the actual truth val here)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    #we're not training here, just using existing model
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            
            # focus only on the last time step AS ONLY WANT NEXT CHARACTER
            logits = logits[:, -1, :] # becomes (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device) #push to GPU

context = torch.zeros((1,1), dtype=torch.long, device=device) #nextline character
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


kYM56?fDUD7;eH[Ol.)]FH﻿0qm4a:]!g)E
AXecv!ZsYK&GuG-fQuf44(I73e7jy'zBH7eR;09M]V?v,t:_L7q4dT-ssR3X m_q-Gag&8TL.prH﻿N7RY﻿n((S jsI6Vy6K9b3M!T_S'U.?YwSIUg(z3uJNhnLgE﻿cb_q﻿QwhUafa*Laa!awrYF 2s"QY8Tx1jP?l8s48LMf1aj:s97C8nqiZs;
OrwOC7XV?qm 97S2GaRbc5Vm﻿9l_5p8a]o.)sr6f4S'
2[Pj3AhVMf5DhHD 5WAH!tE;R6"U42yXf4JLRJZyeg,3He[YmnHer)wwkpDI9U)ppr3ju56,KWdtdvNqC8'O)KDz-7m4GTB
W3q.uGR)LLgrY9*4z9CqlMbNGeX3q;[9gn*BI1&ucPeWzqvNqA-K,G4Pp﻿9s973MdaAES ykoF nk86N
lvfcf2d2Ln?*ZAIBsgEXElm[sb3,PxSF)G4NQY-z'Skc0ZC
 cqHer*x4TFn


In [7]:
a = torch.rand(2, 3, 5)
x, y, z = a.shape
a = a.view(x, y, z)
a.shape

torch.Size([2, 3, 5])

In [ ]:
#pytorch optimizer:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    #sample a batch of data
    xb, yb = get_batch('train') #xb = index / inputs, yb = target

    #evaluate the loss
    logits, loss = model.forward(xb, yb)

    #so previous gradients don't affect current gradient
    #previous gradients are from previous data
    #none instead of 0 as less space (0 would be stored as int64)
    optimizer.zero_grad(set_to_none=True)

    #computes gradients
    loss.backward()

    #updates model's weights based on gradients
    optimizer.step()

print(loss.item())

In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



p4QU*g fLX[q])LFW3ea hosl ing(Ve. edTx7Pplo6j;gre:[F-z,"c&R9J8N9y
JYLa*N?0otwe'QPm"64VhayU&W8REKSle pqGQop1Jgr39se ?p﻿n*GjT'bOmcP8IJ4xqGQ);E952?"a8yo1ST.rm weg;E8'OprekEX-s0;E-z1v,3Wp4hs;X U,&(&&GNqO9C3Xckj53(I78Ghirw08:t zWdiserd EAut'UG8;i9jsljX)?0F( tHr3Kq"z6cA-ett os,js  cP?*V7S2V?.jteiry7-y d lXo,
w(R73uy]ik bis.X[Rpw"hil ss,lftF(ITidvac0,heby, mL2mA9.boinf;R3es,
p
 t!252Xley,!kc0bO.BITOd nyYLs"auwTwIfe? Ojd prste p?)Mcxqf Wo is t
"z,8?:[]DCvugL6se;EHQd Wo ,X&Yzm yVG2(S.xz;ZVQ])L]He3wicwha
